In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Cài đặt PySpark

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()

# Yêu cầu

## Spark Context

In [ ]:
from pyspark import SparkContext
from pyspark.sql import SQLContext




In [ ]:
# sc = SparkContext.getOrCreate("local")
sc = SparkContext("local", "Filter app")
sqlc = SQLContext(sc)

## Đọc dữ liệu data.csv

In [ ]:
data = sc.textFile('/content/data.csv')


## Câu 1: Đếm món hàng

In [ ]:
counters = data.map(lambda row: (','.join(row.split(',')[0:2]), 1))\
            .reduceByKey(lambda x,y: x+y)\
            .map(lambda row: (row[0],str(row[1])))\
            .map(lambda row: (','.join(row))) \
           
t = 0
for ll in counters.collect():
  print(ll)
  t+=1
print(t)

counters.saveAsTextFile('/content/counters')

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
3339,13/04/2015,2
3353,13/04/2015,3
3405,13/04/2015,7
3629,13/04/2015,2
3885,13/04/2015,3
4177,13/04/2015,5
4357,13/04/2015,4
4389,13/04/2015,2
4809,13/04/2015,2
4892,13/04/2015,2
1001,14/04/2015,2
1240,14/04/2015,9
2193,14/04/2015,5
2336,14/04/2015,2
2344,14/04/2015,8
2760,14/04/2015,2
2977,14/04/2015,4
3280,14/04/2015,2
3755,14/04/2015,2
3811,14/04/2015,5
4352,14/04/2015,5
4633,14/04/2015,4
4708,14/04/2015,3
1148,15/04/2015,8
1398,15/04/2015,5
1666,15/04/2015,2
1776,15/04/2015,2
1788,15/04/2015,2
2131,15/04/2015,2
2378,15/04/2015,4
2502,15/04/2015,6
3328,15/04/2015,3
3598,15/04/2015,2
3604,15/04/2015,2
3701,15/04/2015,2
3752,15/04/2015,9
3793,15/04/2015,2
3893,15/04/2015,2
4119,15/04/2015,2
4152,15/04/2015,4
4445,15/04/2015,2
1062,16/04/2015,4
1502,16/04/2015,5
1800,16/04/2015,4
2139,16/04/2015,4
2179,16/04/2015,4
2212,16/04/2015,3
2325,16/04/2015,3
2377,16/04/2015,3
2883,16/04/2015,2
3202,16/04/2015,2
3430,16/04/2015,3
3465,16

Py4JJavaError: ignored

In [ ]:
import os

if os.path.exists('/content/counters/part-00000'):
  sqlc.read.csv('/content/counters/part-00000', header=True).show()
  


+-------------+----------+---+
|Member_number|      Date|  1|
+-------------+----------+---+
|         1249|01/01/2014|  2|
|         1381|01/01/2014|  2|
|         1440|01/01/2014|  2|
|         1659|01/01/2014|  2|
|         1789|01/01/2014|  2|
|         1922|01/01/2014|  2|
|         2226|01/01/2014|  2|
|         2237|01/01/2014|  2|
|         2351|01/01/2014|  2|
|         2542|01/01/2014|  2|
|         2610|01/01/2014|  3|
|         2709|01/01/2014|  2|
|         2727|01/01/2014|  2|
|         2943|01/01/2014|  2|
|         2974|01/01/2014|  3|
|         3681|01/01/2014|  3|
|         3797|01/01/2014|  2|
|         3942|01/01/2014|  3|
|         3956|01/01/2014|  4|
|         4260|01/01/2014|  2|
+-------------+----------+---+
only showing top 20 rows



## Câu 2: Giỏ hàng

In [ ]:
'''
Câu 2
'''
# map: lấy data cột 1 2 và 3 tạo thành 1 tuple dạng ('Member_number;data' ,'itemDescription')
# reduceByKey: để kết hợp 'itemDescription' của dòng x với dùng y
# map: loại bỏ "(", ")"  và chuyển phần tử 2 của tuple thành string
# map :Chuyển tuple thành string kết nối bởi dấu ";"


baskets = data.map(lambda row: (';'.join(row.split(',')[0:2]),row.split(',')[2]))\
                  .reduceByKey(lambda x,y: (x,y))\
                  .map(lambda row: (row[0],(str(row[1]).replace(')','')).replace('(','') ))\
                  .map(lambda row: ";".join(row))
                
for ll in baskets.collect():
  print(ll)
baskets.saveAsTextFile('/content/baskets')

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
3074;13/04/2015;'whole milk', 'margarine'
3339;13/04/2015;'meat', 'margarine'
3353;13/04/2015;'sausage', 'yogurt', 'baking powder'
3405;13/04/2015;'beef', 'ice cream', 'rolls/buns', 'pip fruit', 'whole milk', 'fruit/vegetable juice', 'UHT-milk'
3629;13/04/2015;'chicken', 'tropical fruit'
3885;13/04/2015;'chocolate', 'seasonal products', 'rolls/buns'
4177;13/04/2015;'frankfurter', 'whole milk', 'grapes', 'butter', 'semi-finished bread'
4357;13/04/2015;'sausage', 'chicken', 'canned beer', 'pip fruit'
4389;13/04/2015;'white wine', 'whole milk'
4809;13/04/2015;'zwieback', 'curd'
4892;13/04/2015;'citrus fruit', 'make up remover'
1001;14/04/2015;'beef', 'white bread'
1240;14/04/2015;'frozen vegetables', 'sausage', 'whole milk', 'frozen vegetables', 'rolls/buns', 'other vegetables', 'oil', 'frozen fish', 'baking powder'
2193;14/04/2015;'beef', 'bottled beer', 'specialty chocolate', 'yogurt', 'whole milk'
2336;14/04/2015;'chewing gum', '

In [ ]:
import os

if os.path.exists('/content/baskets/part-00000'):
  sqlc.read.csv('/content/baskets/part-00000', header=True, sep=';').show()

+-------------+----------+--------------------+
|Member_number|      Date|     itemDescription|
+-------------+----------+--------------------+
|         1249|01/01/2014|'citrus fruit', '...|
|         1381|01/01/2014|      'curd', 'soda'|
|         1440|01/01/2014|'other vegetables...|
|         1659|01/01/2014|'specialty chocol...|
|         1789|01/01/2014|'hamburger meat',...|
|         1922|01/01/2014|'tropical fruit',...|
|         2226|01/01/2014|'sausage', 'bottl...|
|         2237|01/01/2014|'bottled water', ...|
|         2351|01/01/2014|'cleaner', 'shopp...|
|         2542|01/01/2014|'sliced cheese', ...|
|         2610|01/01/2014|'hamburger meat',...|
|         2709|01/01/2014|'yogurt', 'frozen...|
|         2727|01/01/2014|'hamburger meat',...|
|         2943|01/01/2014|'whole milk', 'fl...|
|         2974|01/01/2014|'berries', 'whipp...|
|         3681|01/01/2014|'onions', 'whippe...|
|         3797|01/01/2014|'waffles', 'whole...|
|         3942|01/01/2014|'other vegetab

## Câu 3: Tập phổ biến

In [ ]:
'''
Câu 3
'''
import os
# nếu file tồn tại
if os.path.exists('/content/baskets/part-00000'):
  # đọc file
  dfBaskets = sqlc.read.csv('/content/baskets/part-00000', header=True, sep=';')

# .rdd => chuyển dataframe thành rDD
# .map => tạo tuple với dạng (Member_number,Data,list), 
# trong đó list item: xóa kí tự " ' ", sử dụng split(",") để lai list, dùng set() để loại bỏ kí tự giống, rồi ép kiểu list lại
# .map => xử lí list, xóa các kí tự rỗng trên mỗi x thuộc list
# .toDF => chuyển rDD sang DF
# .withColumnRenamed => đổi tên cột
dfBaskets= dfBaskets.rdd\
          .map(lambda row: row)\
          .map(lambda row: (row[0],row[1],list(set(row[2].replace('\'','').split(','))))) \
         .map(lambda row: (row[0],row[1], list(set([x.strip() for x in row[2]]))))\
         .toDF()\
         .withColumnRenamed('_1', 'Member_number') \
        .withColumnRenamed('_2', 'Date') \
        .withColumnRenamed('_3', 'Items') \


         
dfBaskets.show()

+-------------+----------+--------------------+
|Member_number|      Date|               Items|
+-------------+----------+--------------------+
|         1249|01/01/2014|[coffee, citrus f...|
|         1381|01/01/2014|        [soda, curd]|
|         1440|01/01/2014|[yogurt, other ve...|
|         1659|01/01/2014|[specialty chocol...|
|         1789|01/01/2014|[hamburger meat, ...|
|         1922|01/01/2014|[other vegetables...|
|         2226|01/01/2014|[sausage, bottled...|
|         2237|01/01/2014|[bottled water, I...|
|         2351|01/01/2014|[shopping bags, c...|
|         2542|01/01/2014|[bottled water, s...|
|         2610|01/01/2014|[hamburger meat, ...|
|         2709|01/01/2014|[yogurt, frozen v...|
|         2727|01/01/2014|[hamburger meat, ...|
|         2943|01/01/2014|[flower seeds, wh...|
|         2974|01/01/2014|[berries, whipped...|
|         3681|01/01/2014|[onions, whipped/...|
|         3797|01/01/2014|[whole milk, waff...|
|         3942|01/01/2014|[yogurt, other

In [ ]:
from pyspark.ml.fpm import FPGrowth
fpGrowth = FPGrowth(itemsCol="Items", 
                    minSupport=0.01, 
                    minConfidence=0.1)
model = fpGrowth.fit(dfBaskets)

In [ ]:
 # Display frequent itemsets.
model.freqItemsets.show()

+--------------------+----+
|               items|freq|
+--------------------+----+
|              [beef]| 508|
|             [sugar]| 265|
|               [oil]| 223|
|         [chocolate]| 353|
|        [white wine]| 175|
|             [candy]| 215|
|  [processed cheese]| 152|
|              [meat]| 252|
|       [frankfurter]| 565|
|           [sausage]| 903|
|           [berries]| 326|
|            [coffee]| 473|
|         [pip fruit]| 734|
|  [hygiene articles]| 205|
|       [salty snack]| 281|
|       [white bread]| 359|
|[specialty chocol...| 239|
|     [domestic eggs]| 555|
|        [rolls/buns]|1646|
|[rolls/buns, othe...| 158|
+--------------------+----+
only showing top 20 rows



In [ ]:
model.associationRules.show()

+------------------+------------+-------------------+------------------+--------------------+
|        antecedent|  consequent|         confidence|              lift|             support|
+------------------+------------+-------------------+------------------+--------------------+
|[other vegetables]|[whole milk]|0.12151067323481117|0.7694304712706219|0.014836596939116486|
|          [yogurt]|[whole milk]|0.12996108949416343|0.8229402378760761|0.011160863463209249|
|      [rolls/buns]|[whole milk]|0.12697448359659783|0.8040284376030019|0.013967787208447505|
|            [soda]|[whole milk]|0.11975223675154852|0.7582956912879478|0.011628684087415625|
+------------------+------------+-------------------+------------------+--------------------+



## Câu 4: Giỏ hàng thành vectors 

In [ ]:
'''
Câu 4
'''
# đọc file
dfMembers = sqlc.read.option("header", True) \
          .option("delimiter", ",") \
          .option("inferSchema", True) \
          .csv('/content/data.csv')

def groupBy(x,y):
  string = ''
  string = string +str(x)+","+str(y)
  temp = list(set(string.split(',')))
  string = ','.join(temp)
  return string

dfMembers = dfMembers.rdd\
             .map(lambda row: (row[0],row[2]))\
             .reduceByKey(groupBy)\
             .toDF()\
             .withColumnRenamed('_1', 'Member_number') \
              .withColumnRenamed('_2', 'Items') \

dfMembers.show(truncate=False)
# for i in dfMembers.collect():
#   print(i)


+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|Member_number|Items                                                                                                                                                                                                                       |
+-------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1249         |coffee,citrus fruit,bottled beer,frozen meals,soda                                                                                                                                                                          |
|1381         |coffee,UHT-milk,other vegetables,dome

In [ ]:
from pyspark.ml.linalg import Vectors
items = sorted(dfMembers.rdd\
                .map(lambda x: (set(x[1].split(','))))\
                .reduce(lambda x,y: list(set(x)| set(y))))\
     
print(sorted(items))


dictItems ={}
for i in range(len(items)):
  dictItems[items[i]] =i
print("dictItems= " ,dictItems)


def basket2vector(member, basket, dictItems):
  lenItem = len(dictItems)
  indexItem = {}
  for i in basket.split(','):
    indexItem[dictItems[i]] = 1 
  return Vectors.sparse(lenItem,indexItem)

key = basket2vector(dfMembers.first()['Member_number'],dfMembers.first()['Items'],dictItems)

['Instant food products', 'UHT-milk', 'abrasive cleaner', 'artif. sweetener', 'baby cosmetics', 'bags', 'baking powder', 'bathroom cleaner', 'beef', 'berries', 'beverages', 'bottled beer', 'bottled water', 'brandy', 'brown bread', 'butter', 'butter milk', 'cake bar', 'candles', 'candy', 'canned beer', 'canned fish', 'canned fruit', 'canned vegetables', 'cat food', 'cereals', 'chewing gum', 'chicken', 'chocolate', 'chocolate marshmallow', 'citrus fruit', 'cleaner', 'cling film/bags', 'cocoa drinks', 'coffee', 'condensed milk', 'cooking chocolate', 'cookware', 'cream', 'cream cheese ', 'curd', 'curd cheese', 'decalcifier', 'dental care', 'dessert', 'detergent', 'dish cleaner', 'dishes', 'dog food', 'domestic eggs', 'female sanitary products', 'finished products', 'fish', 'flour', 'flower (seeds)', 'flower soil/fertilizer', 'frankfurter', 'frozen chicken', 'frozen dessert', 'frozen fish', 'frozen fruits', 'frozen meals', 'frozen potato products', 'frozen vegetables', 'fruit/vegetable juic

In [ ]:
dfMembers= dfMembers.rdd\
        .map(lambda row: (row[0],basket2vector(row[0],row[1],dictItems)))\
        .toDF() \
        .withColumnRenamed('_1', 'Member_number') \
        .withColumnRenamed('_2', 'Items') \

dfMembers.show()



+-------------+--------------------+
|Member_number|               Items|
+-------------+--------------------+
|         1249|(167,[11,30,34,61...|
|         1381|(167,[1,10,11,28,...|
|         1440|(167,[28,64,102,1...|
|         1659|(167,[12,14,26,34...|
|         1789|(167,[8,18,30,58,...|
|         1922|(167,[10,12,15,16...|
|         2226|(167,[9,12,23,40,...|
|         2237|(167,[0,12,27,34,...|
|         2351|(167,[31,49,75,10...|
|         2542|(167,[12,82,88,94...|
|         2610|(167,[11,49,68,90...|
|         2709|(167,[12,30,40,44...|
|         2727|(167,[11,52,56,61...|
|         2943|(167,[20,29,33,54...|
|         2974|(167,[9,12,63,70,...|
|         3681|(167,[47,63,88,99...|
|         3797|(167,[15,38,64,10...|
|         3942|(167,[0,21,32,35,...|
|         3956|(167,[0,16,21,28,...|
|         4260|(167,[14,20,27,63...|
+-------------+--------------------+
only showing top 20 rows



## Câu 5: Giỏ hàng tương tự

In [ ]:
'''
Câu 5
'''
from pyspark.ml.feature import MinHashLSH
mh = MinHashLSH(inputCol="Items", outputCol="hashes", numHashTables=5)
model = mh.fit(dfMembers)

In [ ]:
model.transform(dfMembers).show()


+-------------+--------------------+--------------------+
|Member_number|               Items|              hashes|
+-------------+--------------------+--------------------+
|         1249|(167,[11,30,34,61...|[[8.3690003E7], [...|
|         1381|(167,[1,10,11,28,...|[[8.3453121E7], [...|
|         1440|(167,[28,64,102,1...|[[2.96379232E8], ...|
|         1659|(167,[12,14,26,34...|[[1.75113252E8], ...|
|         1789|(167,[8,18,30,58,...|[[6.00965474E8], ...|
|         1922|(167,[10,12,15,16...|[[2.96379232E8], ...|
|         2226|(167,[9,12,23,40,...|[[1.1424338E8], [...|
|         2237|(167,[0,12,27,34,...|[[2.66299619E8], ...|
|         2351|(167,[31,49,75,10...|[[8.4420808E8], [...|
|         2542|(167,[12,82,88,94...|[[5.3985872E8], [...|
|         2610|(167,[11,49,68,90...|[[1.14006498E8], ...|
|         2709|(167,[12,30,40,44...|[[2.96616114E8], ...|
|         2727|(167,[11,52,56,61...|[[8.3690003E7], [...|
|         2943|(167,[20,29,33,54...|[[2.96379232E8], ...|
|         2974

In [ ]:
from pyspark.sql.functions import col
print("Approximately joining dfA and dfB on distance smaller than 0.3:")
model.approxSimilarityJoin(dfMembers,dfMembers, 0.3, distCol="JaccardDistance")\
    .select(col("datasetA.Member_number").alias("idA"),
            col("datasetB.Member_number").alias("idB"),
            col("JaccardDistance"))\
    .where(col("JaccardDistance")>0)\
    .show()

          

Approximately joining dfA and dfB on distance smaller than 0.3:
+----+----+-------------------+
| idA| idB|    JaccardDistance|
+----+----+-------------------+
|4174|2199|               0.25|
|2347|1401|               0.25|
|4805|3216|               0.25|
|2360|4060| 0.2857142857142857|
|4805|3715|               0.25|
|3714|2911|               0.25|
|1643|4535|0.19999999999999996|
|3216|4805|               0.25|
|2911|3714|               0.25|
|3008|3498| 0.2857142857142857|
|4060|2360| 0.2857142857142857|
|1367|1161| 0.2857142857142857|
|4535|1643|0.19999999999999996|
|4730|1164|0.19999999999999996|
|3124|4174|               0.25|
|4342|1056| 0.2857142857142857|
|2088|4116| 0.2857142857142857|
|1164|4730|0.19999999999999996|
|1976|2342|               0.25|
|1161|1367| 0.2857142857142857|
+----+----+-------------------+
only showing top 20 rows



In [ ]:

print("key = ", key)
model.approxNearestNeighbors(dfMembers,key, 5).show()

key =  (167,[11,30,34,61,138],[1.0,1.0,1.0,1.0,1.0])
+-------------+--------------------+--------------------+------------------+
|Member_number|               Items|              hashes|           distCol|
+-------------+--------------------+--------------------+------------------+
|         1249|(167,[11,30,34,61...|[[8.3690003E7], [...|               0.0|
|         1321|(167,[11,30,138],...|[[8.74761457E8], ...|               0.4|
|         1263|(167,[11,30,61,10...|[[8.3690003E7], [...|               0.5|
|         2708|(167,[3,34,61,75,...|[[2.3057013E7], [...|0.5714285714285714|
|         4327|(167,[30,34,63,76...|[[2.66299619E8], ...|0.5714285714285714|
+-------------+--------------------+--------------------+------------------+



## Câu 6: Phân cụm người dùng theo giỏ hàng

In [ ]:
'''
Câu 6
'''


from pyspark.ml.clustering import KMeans, KMeansModel

kmeans = KMeans(k=5)
kmeans.setSeed(1)
kmeans.setWeightCol("Member_number")
kmeans.setMaxIter(10)





KMeans_d26842d7429c

In [ ]:
print(kmeans.getMaxIter())
kmeans.clear(kmeans.maxIter)
dfMembers.printSchema()


10
root
 |-- Member_number: long (nullable = true)
 |-- Items: vector (nullable = true)



In [ ]:
transformed = model.transform(dfMembers.withColumnRenamed('Items', 'features')).select("Member_number","features", "prediction")


In [ ]:
transformed.show()


+-------------+--------------------+----------+
|Member_number|            features|prediction|
+-------------+--------------------+----------+
|         1249|(167,[11,30,34,61...|         1|
|         1381|(167,[1,10,11,28,...|         0|
|         1440|(167,[28,64,102,1...|         4|
|         1659|(167,[12,14,26,34...|         4|
|         1789|(167,[8,18,30,58,...|         3|
|         1922|(167,[10,12,15,16...|         4|
|         2226|(167,[9,12,23,40,...|         4|
|         2237|(167,[0,12,27,34,...|         1|
|         2351|(167,[31,49,75,10...|         1|
|         2542|(167,[12,82,88,94...|         3|
|         2610|(167,[11,49,68,90...|         1|
|         2709|(167,[12,30,40,44...|         4|
|         2727|(167,[11,52,56,61...|         2|
|         2943|(167,[20,29,33,54...|         2|
|         2974|(167,[9,12,63,70,...|         1|
|         3681|(167,[47,63,88,99...|         2|
|         3797|(167,[15,38,64,10...|         4|
|         3942|(167,[0,21,32,35,...|    